Задание номер 1. Баессовский
классификатор

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("names.csv")
df

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy
...,...,...,...,...
257995,2008,Carleigh,0.000128,girl
257996,2008,Iyana,0.000128,girl
257997,2008,Kenley,0.000127,girl
257998,2008,Sloane,0.000127,girl


разбиваем данные на test и train с тестовым размером 30% и добавляем перемешивание

In [ ]:
X = df.drop(columns=['sex'])
y = df["sex"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from collections import defaultdict
from math import log

def train(samples):
    """
    считаем частоту классов и букв при условии определенного класса
    """
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0)
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        freq[label, feats] += 1          # count features frequencies

    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)
    return classes, freq, len(samples)                    # return P(C) and P(O|C)

def classify(classifier, name):
    """
    классифицируем объект с добавленным смещением Лапласса при условии 0 частоты буквы
    """
    classes, prob, dlina = classifier
    V_set = set()
    for key in prob.keys():
        V_set.add(key[1])
    V = len(V_set)
    return min(classes.keys(),              # calculate argmin(-log(P(C|O))) -> argmax(P(C|O))
        key = lambda cl: -log(classes[cl]) \
               + sum(-log(((prob.get((cl,feat), 0) * dlina * classes[cl]) + 1) \
                          / (V + (dlina * classes[cl]))) for feat in name))

def get_features(sample):
    return (sample[-1]) # get last letter


In [ ]:
new_df_train = [(get_features(X_train.name.iloc[i]), y_train.iloc[i]) for i in range(len(X_train))]
new_df_train

In [ ]:
model = train(new_df_train)

In [ ]:
result = []
for i in range(len(X_test)):
    temp = classify(model, get_features(X_test.name.iloc[i]))
    result.append([y_test.iloc[i], temp])
result = np.array(result)


средняя точность(accuracy)


In [ ]:
res = 0
for i in result:
  if i[0] == i[1]:
    res += 1
mean_acur = res/len(result)
mean_acur

0.7801162790697674

считаем две доп метрики precision и recall с предсказывемым классом-"boy"


In [ ]:
from sklearn.metrics import precision_score, recall_score
print("precision: ",precision_score(result[:, 0], result[:, 1], pos_label='boy'))
print("Recall: ",recall_score(result[:, 0], result[:, 1], pos_label='boy'))

precision:  0.7486732982609295
Recall:  0.8427899037964208


пробуем изменять функцию, которая брала последнюю букву
1 попытка взять целое слово


In [ ]:
def get_features(sample): return sample # get last letter

In [ ]:
new_df_train1 = [(get_features(X_train.name.iloc[i]), y_train.iloc[i]) for i in range(len(X_train))]
model_tun1 = train(new_df_train1)

средняя точность затюненной модели

In [ ]:
result = []
for i in range(len(X_test)):
    temp = classify(model, get_features(X_test.name.iloc[i]))
    result.append([y_test.iloc[i], temp])
result = np.array(result)
res = 0
for i in result:
  if i[0] == i[1]:
    res += 1
mean_acur = res/len(result)
mean_acur

0.655

новые precision и recall

In [ ]:
print("precision: ",precision_score(result[:, 0], result[:, 1], pos_label='boy'))
print("Recall: ",recall_score(result[:, 0], result[:, 1], pos_label='boy'))

precision:  0.6245809125174403
Recall:  0.7756542877831799


In [ ]:
def classify(classifier, feats):
  classes, prob, dlina = classifier
  V_set = set()
  for key in prob.keys():
      V_set.add(key[1])
  V = len(V_set)
  def pravda(cl):
    temp = 1
    for feat in feats:
      temp *= ((prob.get((cl,feat), 0) * dlina * classes[cl]) + 1) / (V + (dlina * classes[cl]))
    return temp*classes[cl]
  return max(classes.keys(),              # calculate argmin(-log(P(C|O))) -> argmax(P(C|O))
      key = lambda cl: pravda(cl))

попробуем обучить на новом classify(и увидим что ничего не поменяется, результаты дальше будуть абсолютно аналогичные)

In [ ]:
result = []
for i in range(len(X_test)):
    temp = classify(model, get_features(X_test.name.iloc[i]))
    result.append([y_test.iloc[i], temp])
result = np.array(result)
res = 0
for i in result:
  if i[0] == i[1]:
    res += 1
mean_acur = res/len(result)
mean_acur

0.655

Импортируем гауссовский и мультиномиальный наивный баесс(они отличаются ядерными функциями)

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [ ]:
df = pd.read_csv("names.csv")
df

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy
...,...,...,...,...
257995,2008,Carleigh,0.000128,girl
257996,2008,Iyana,0.000128,girl
257997,2008,Kenley,0.000127,girl
257998,2008,Sloane,0.000127,girl


In [ ]:
from sklearn.utils import shuffle

первый подход заключается попробовать применить onehotencoder(если вкратце n-мерные базисные векторы , где n-размерность(кол-во уникальных букв), а 1 ставим на букве в исходном столбце
) к последней букве имени


In [ ]:
new_names = [get_features(df.iloc[i,1]) for i in range(len(df))]
df["name"] = new_names
df

In [ ]:
df_new = pd.get_dummies(df, columns=["name"])
df_new

,year,percent,sex,name_a,name_b,name_c,name_d,name_e,name_f,name_g,...,name_q,name_r,name_s,name_t,name_u,name_v,name_w,name_x,name_y,name_z
0,1880,0.081541,boy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1880,0.080511,boy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1880,0.050057,boy,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1880,0.045167,boy,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1880,0.043292,boy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257995,2008,0.000128,girl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257996,2008,0.000128,girl,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257997,2008,0.000127,girl,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
257998,2008,0.000127,girl,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
target = df_new.iloc[:, 2]
X = df_new.drop(columns=["sex"])
X

,year,percent,name_a,name_b,name_c,name_d,name_e,name_f,name_g,name_h,...,name_q,name_r,name_s,name_t,name_u,name_v,name_w,name_x,name_y,name_z
0,1880,0.081541,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1880,0.080511,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1880,0.050057,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1880,0.045167,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1880,0.043292,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257995,2008,0.000128,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
257996,2008,0.000128,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257997,2008,0.000127,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
257998,2008,0.000127,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

второй подход это применить labelencoder(у каждой буквы свой порядок)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df["name"] = encoder.fit_transform(df["name"])

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.3, random_state=42)

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)
np.mean(y_test==y_predicted)

0.6420671834625323

In [ ]:
clf = GaussianNB()
clf.fit(X_train1, y_train1)
y_predicted = clf.predict(X_test1)
np.mean(y_test==y_predicted)

0.5219638242894057

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

print(np.mean(y_test==y_predicted))

0.7801162790697674


In [ ]:
clf = MultinomialNB()
clf.fit(X_train1, y_train1)
y_predicted = clf.predict(X_test1)

print(np.mean(y_test==y_predicted))

0.5284237726098191


задание 2: ирисы фишера и LDA

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

применяем наш LDA который по сути находит первые k собственных векторов(которыми описывается все пространтсво) , чтобы в последствии сократить размерность исходного пространства. Таким образом мы урезаем общую дисперсию данных


In [ ]:
def LDA_dimensionality(X, y, k):
    '''
    X - набор данных, y - метка, k - целевой размер
    '''
    label_ = list(set(y))

    X_classify = {}

    for label in label_:
        X1 = np.array([X[i] for i in range(len(X)) if y[i] == label])
        X_classify[label] = X1

    mju = np.mean(X, axis=0)
    mju_classify = {}

    for label in label_:
        mju1 = np.mean(X_classify[label], axis=0)
        mju_classify[label] = mju1

    #St = np.dot((X - mju).T, X - mju)

    Sw = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sw += np.dot((X_classify[i] - mju_classify[i]).T,
                     X_classify[i] - mju_classify[i])

    # Sb=St-Sw

    Sb = np.zeros((len(mju), len(mju)))  # Вычислить матрицу внутриклассовой дивергенции
    for i in label_:
        Sb += len(X_classify[i]) * np.dot((mju_classify[i] - mju).reshape(
            (len(mju), 1)), (mju_classify[i] - mju).reshape((1, len(mju))))

    eig_vals, eig_vecs = np.linalg.eig(
        np.linalg.inv(Sw).dot(Sb))  # Вычислить собственное значение и собственную матрицу Sw-1 * Sb

    sorted_indices = np.argsort(eig_vals)
    topk_eig_vecs = eig_vecs[:, sorted_indices[:-k - 1:-1]]  # Извлекаем первые k векторов признаков
    return topk_eig_vecs

In [ ]:
W = LDA_dimensionality(X_train, y_train, 3)
X_new = np.dot(X_train, W)
X_new

In [ ]:
X_test_new = np.dot(X_test, W)

In [ ]:
label_ = list(set(y))

X_classify = {}

for label in label_:
    X1 = np.array([X_new[i] for i in range(len(X_new)) if y[i] == label])
    X_classify[label] = X1

mju_classify = {}

for label in label_:
    mju1 = np.mean(X_classify[label], axis=0)
    mju_classify[label] = mju1
mju_classify

{0: array([-0.26199565+0.j        , -2.30323114+0.j        ,
         1.32823711-0.67126689j]),
 1: array([-0.41198139+0.j        , -2.30964051+0.j        ,
         1.32603108-0.67764708j]),
 2: array([-0.40115555+0.j        , -2.31694667+0.j        ,
         1.28639251-0.69691638j])}

написал свой "игрушечный" predict, который классифицирует объект классом, котрый ближе всчего к "центроиду" соответсвущего класса

In [ ]:
def predict(X):
  result = []
  for i in X:
    a = np.linalg.norm(i - mju_classify[0])
    b = np.linalg.norm(i - mju_classify[1])
    c = np.linalg.norm(i - mju_classify[2])
    res = min(a, min(b, c))
    if res == a:
      result.append(0)
    elif res == b:
      result.append(1)
    elif res == c:
      result.append(2)
  return np.array(result)

In [ ]:
y_pred = predict(X_test_new)

In [ ]:
y_pred

array([1, 0, 1, 1, 1, 0, 2, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       0])

In [ ]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7333333333333333

применим LDA из sklearn

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
y_pred1= clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred1)

1.0

модифицируем параметры

In [ ]:
clf = LinearDiscriminantAnalysis(solver="lsqr", n_components=2)
clf.fit(X_train, y_train)
y_pred1= clf.predict(X_test)
accuracy_score(y_test, y_pred1)

1.0

Задание 3: KNN

In [ ]:
from sklearn.datasets import load_wine

In [ ]:
wine_data = load_wine()

In [ ]:
df = pd.DataFrame(data=wine_data.data, columns=wine_data.feature_names)
df['target'] = wine_data.target
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X=df.iloc[:, :-1]
Y=df.iloc[:, -1]

будем делать кросс валидцию при этом используя в цикле , а именно GridSearch так это делает, разные параметры, чтобы подобрать лучшую модель на нестандартизированных данных

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
par = {
    "n_neighbors":np.arange(1, 51, 1)
}
loo = LeaveOneOut()
model1 = GridSearchCV(KNeighborsClassifier(), par, cv=KFold(3))
model2 = GridSearchCV(KNeighborsClassifier(), par, cv=StratifiedKFold(3))
model3 = GridSearchCV(KNeighborsClassifier(), par, cv=loo)
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

GridSearchCV(cv=LeaveOneOut(), estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])})

смотрим accuracy

In [ ]:

print(accuracy_score(y_test, model1.predict(X_test)))
print(accuracy_score(y_test, model2.predict(X_test)))
print(accuracy_score(y_test, model3.predict(X_test)))

0.7962962962962963
0.7962962962962963
0.7962962962962963


In [ ]:
model3.best_estimator_

KNeighborsClassifier(n_neighbors=1)

In [ ]:
from sklearn.preprocessing import scale

делаем все тоже самое на стандратизированных данных

In [ ]:
X_new = scale(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.3, random_state=42)

In [ ]:
par = {
    "n_neighbors":np.arange(1, 51, 1)
}
loo = LeaveOneOut()
model1 = GridSearchCV(KNeighborsClassifier(), par, cv=KFold(3))
model2 = GridSearchCV(KNeighborsClassifier(), par, cv=StratifiedKFold(3))
model3 = GridSearchCV(KNeighborsClassifier(), par, cv=loo)
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

GridSearchCV(cv=LeaveOneOut(), estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])})

в этот раз 3-я модель стала лучший и данный пример показывает всю важность препроцессинага :))))


In [ ]:
print(accuracy_score(y_test, model1.predict(X_test)))
print(accuracy_score(y_test, model2.predict(X_test)))
print(accuracy_score(y_test, model3.predict(X_test)))

0.9597367402245451
0.9597367402245451
0.967741935483871


In [ ]:
model3.best_estimator_

KNeighborsClassifier(n_neighbors=16)